In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#Loading The Dataset
true=pd.read_csv('True.csv')
fake=pd.read_csv('Fake.csv')
true['label']=0
fake['label']=1
#Data Merging
data1=true[['text','label']]
data2=fake[['text','label']]
dataset=pd.concat([data1,data2])
dataset.shape

(44898, 2)

In [2]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [3]:
dataset['label'].value_counts()

label
1    23481
0    21417
Name: count, dtype: int64

In [4]:
#using stopwords method
ps=WordNetLemmatizer()
stopwords=stopwords.words('english')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
#cleaning Data
def clean_row(row):
    row=row.lower()
    row=re.sub('[^a-zA-Z]',' ',row)

    tokens=row.split()
    news=[ps.lemmatize(word) for word in tokens if not word in stopwords]
    cleaned_news = ' '.join(news)
    return cleaned_news

dataset['text']=dataset['text'].apply(lambda x : clean_row(x))
vectorize=TfidfVectorizer(max_features = 5000 ,lowercase=False,ngram_range=(1,2))
x=dataset.iloc[:30000,0]
y=dataset.iloc[:30000,1]


In [6]:
#datasplitting
train_data,test_data,train_label,test_label=train_test_split(x,y,test_size=0.2,random_state=0)
#textvectorizing
vec_train_data=vectorize.fit_transform(train_data)
vec_train_data=vec_train_data.toarray()
vec_test_data=vectorize.fit_transform(test_data)
vec_test_data=vec_test_data.toarray()
training_data=pd.DataFrame(vec_train_data,columns= vectorize.get_feature_names_out())
testing_data=pd.DataFrame(vec_test_data,columns= vectorize.get_feature_names_out())
clf=MultinomialNB()
clf.fit(training_data,train_label)

MultinomialNB()

In [7]:
y_pred=clf.predict(testing_data)
y_pred_train=clf.predict(training_data)
print(accuracy_score(train_label,y_pred_train))

0.9750833333333333


In [8]:
txt=''# Put news here
news=clean_row(txt)
pred=clf.predict(vectorize.transform([news]).toarray())
if pred==0:
    print('True news')
else:
    print('Fake news')

True news


c:\Users\PC\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(
